In [1]:
pip install PyMuPDF tensorflow Pillow


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [8]:
import fitz  # PyMuPDF para manipular PDFs
import os
import random
from shutil import copyfile

# Função para extrair imagens de um arquivo PDF usando PyMuPDF e salvar como PNG
def extract_images_from_pdf(pdf_path, output_dir):
    doc = fitz.open(pdf_path)
    images = []
    for page_num in range(len(doc)):
        page = doc.load_page(page_num)
        img = page.get_pixmap()
        img_path = os.path.join(output_dir, f"{os.path.basename(pdf_path)}_page{page_num+1}.png")
        img.writePNG(img_path)
        images.append(img_path)
    doc.close()
    return images

# Diretórios originais dos PDFs de acertos e erros
dir_acertos = 'datasets/acertos'
dir_erros = 'datasets/erros'

# Diretórios para imagens de treino e validação
output_dir_acertos_treino = 'datasets/acertos/treino'
output_dir_acertos_validacao = 'datasets/acertos/validacao'
output_dir_erros_treino = 'datasets/erros/treino'
output_dir_erros_validacao = 'datasets/erros/validacao'

# Criar diretórios de saída se não existirem
os.makedirs(output_dir_acertos_treino, exist_ok=True)
os.makedirs(output_dir_acertos_validacao, exist_ok=True)
os.makedirs(output_dir_erros_treino, exist_ok=True)
os.makedirs(output_dir_erros_validacao, exist_ok=True)

# Função para dividir imagens em treino e validação
def split_train_validation(images, output_dir_train, output_dir_valid, validation_split=0.2):
    random.shuffle(images)
    split_index = int(len(images) * (1 - validation_split))
    for i, img_path in enumerate(images):
        if i < split_index:
            copyfile(img_path, os.path.join(output_dir_train, os.path.basename(img_path)))
        else:
            copyfile(img_path, os.path.join(output_dir_valid, os.path.basename(img_path)))

# Extrair imagens dos PDFs de acertos
acertos_images = []
for filename in os.listdir(dir_acertos):
    if filename.endswith(".pdf"):
        pdf_path = os.path.join(dir_acertos, filename)
        acertos_images.extend(extract_images_from_pdf(pdf_path, output_dir_acertos_treino))

# Dividir imagens de acertos em treino e validação
split_train_validation(acertos_images, output_dir_acertos_treino, output_dir_acertos_validacao)

# Extrair imagens dos PDFs de erros
erros_images = []
for filename in os.listdir(dir_erros):
    if filename.endswith(".pdf"):
        pdf_path = os.path.join(dir_erros, filename)
        erros_images.extend(extract_images_from_pdf(pdf_path, output_dir_erros_treino))

# Dividir imagens de erros em treino e validação
split_train_validation(erros_images, output_dir_erros_treino, output_dir_erros_validacao)


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Definindo parâmetros
img_width, img_height = 150, 150
batch_size = 16
epochs = 10

# Diretórios para dados de treino e validação
train_data_dir = 'datasets/acertos_erros/treino'
validation_data_dir = 'datasets/acertos_erros/validacao'

# Criando geradores de dados para treino e validação
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary'  # Como temos duas classes, usamos class_mode='binary'
)

validation_generator = train_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary'
)


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

# Criando o modelo CNN
model = Sequential()

model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(img_width, img_height, 3)))
model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))

model.add(Flatten())

model.add(Dense(512, activation='relu'))
model.add(Dense(1, activation='sigmoid'))  # Saída binária (sigmoid para classificação binária)

# Compilando o modelo
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Treinando o modelo
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size
)
